# Plot meshing and boundaries on the Earth

In [ ]:
import numpy as np     
import plotly.graph_objects as go
import gstlearn.plot3D as gop
import IPython
import os
from numpy import pi, cos, sin
import gstlearn as gl 

Definition of the Meshing

In [ ]:
gl.variety_define(1)

mesh = gl.MeshSpherical()
err = mesh.reset(None,None,triswitch = "-r4",verbose=False)

Display a white skin around the meshing

In [ ]:
blank = gop.SurfaceOnMesh(mesh)

In [ ]:
fig = go.Figure(data = [blank])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
fig.show()

We overlay the meshing

In [ ]:
meshing = gop.Meshing(mesh)

In [ ]:
fig = go.Figure(data = [blank, meshing])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
fig.show()

Drawing a polygon (we use the one containing the land boundaries)

In [ ]:
name = os.path.join(os.getenv('GSTLEARN_DATA'), "boundaries", "world.poly")
print(name)
poly = gl.Polygons.createFromNF(name)

In [ ]:
name

In [ ]:
boundaries = gop.PolygonOnSphere(poly)
equator = gop.Equator(width=5)
meridians = gop.Meridians(angle=20,color='blue')
parallels = gop.Parallels(angle=30,color='red')
pole = gop.Pole()
poleaxis = gop.PolarAxis()

In [ ]:
fig = go.Figure(data = [blank,boundaries,equator,meridians,parallels,pole,poleaxis])
fig.show()